In [5]:
# Imports
import spacy as spc
import pandas as pd
import math as mt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchtext as tt

In [31]:
# Helper


In [26]:
# Model init
class CNN(nn.Module):
  def __init__(self, input_dims, numOfKernels, numOfNeurons, kernelSize, numOfConvLayers, batchNorm):
    super(CNN, self).__init__()         
    self.numOfKernels = numOfKernels
    self.batchNorm = batchNorm
    self.numOfConvLayers = numOfConvLayers

    # Convolutional Layers
    self.conv1 = nn.Conv2d(3,numOfKernels, kernelSize)
    self.conv2 = nn.Conv2d(numOfKernels,numOfKernels, kernelSize)
    self.conv_BN = nn.BatchNorm2d(numOfKernels)

    # Determine the output size after the convolutional layer
    fullLayerSize_x = input_dims[1]
    fullLayerSize_y = input_dims[0]
    for i in range (self.numOfConvLayers):
      fullLayerSize_x = (fullLayerSize_x-kernelSize+1)//2
      fullLayerSize_y = (fullLayerSize_y-kernelSize+1)//2

    # Error check the output size
    if fullLayerSize_x <= 0 or fullLayerSize_y <= 0:
      raise Exception("Too many convolutional layer for the input size, please decrease numOfConvLayers.")

    # Fully connected layers
    self.fc1 = nn.Linear(numOfKernels*fullLayerSize_x*fullLayerSize_y, numOfNeurons)
    self.fc1_BN = nn.BatchNorm1d(numOfNeurons)
    self.pool = nn.MaxPool2d(2,2)
    self.fc2 = nn.Linear(numOfNeurons, 6)
    self.fc2_BN = nn.BatchNorm1d(6)

  def forward(self, x):
    if self.batchNorm == True:
      x = self.pool(activation(self.conv_BN(self.conv1(x))))
      for i in range (self.numOfConvLayers - 1):
        x = self.pool(activation(self.conv_BN(self.conv2(x))))
      x = x.view(-1, x.shape[1]*x.shape[2]*x.shape[3])
      x = activation(self.fc1_BN(self.fc1(x)))
      x = self.fc2_BN(self.fc2(x))
    else: 
      x = self.pool(activation(self.conv1(x)))
      for i in range (self.numOfConvLayers - 1):
        x = self.pool(activation(self.conv2(x)))
      x = x.view(-1, x.shape[1]*x.shape[2]*x.shape[3])
      x = activation(self.fc1(x))
      x = self.fc2(x)
    return x


In [11]:
# Load Parameter Dict
mdl = torch.load('mdl.pt')

In [39]:
# Game Loop
while True:
    print("Enter a sentence")
    query = input()
    if query == '~~': # For testing and overall friendliness
        break
    if len(query.split(' ')) < 4:
        print("Please input a longer sentence... (4 word minimum)")
        continue
    print(query)
    print()
    sentVec = [vocab.stoi[tok] for tok in tokenizer(query)]
    sentTensor = torch.LongTensor(sentVec).view(-1, 1)
    predbase = float(F.sigmoid(base(sentTensor)))
    predrnn = float(F.sigmoid(rnn(sentTensor)[0]))
    predcnn = float(F.sigmoid(cnn(sentTensor)))
    print("Model baseline: {} ({})".format(isObjective(predbase), round(predbase, 3)))
    print("Model rnn: {} ({})".format(isObjective(predrnn), round(predrnn, 3)))
    print("Model cnn: {} ({})".format(isObjective(predcnn), round(predcnn, 3)))
    print()

Enter a sentence
Please input a longer sentence... (4 word minimum)
Enter a sentence
Please input a longer sentence... (4 word minimum)
Enter a sentence
there is my dad

Model baseline: subjective (0.597)
Model rnn: objective (0.006)
Model cnn: objective (0.08)

Enter a sentence
An example output on the console is given below

Model baseline: subjective (0.595)
Model rnn: objective (0.034)
Model cnn: objective (0.328)

Enter a sentence
I had a mental breakdown last week

Model baseline: objective (0.077)
Model rnn: objective (0.002)
Model cnn: objective (0.022)

Enter a sentence
Please input a longer sentence... (4 word minimum)
Enter a sentence
Please input a longer sentence... (4 word minimum)
Enter a sentence
Please input a longer sentence... (4 word minimum)
Enter a sentence
Please input a longer sentence... (4 word minimum)
Enter a sentence
Please input a longer sentence... (4 word minimum)
Enter a sentence
Please input a longer sentence... (4 word minimum)
Enter a sentence


KeyboardInterrupt: Interrupted by user